## Модель Transformer-1
Задание
1. Взять предобученную трансформерную архитектуру и решить задачу перевода (для тогоже корпуса что вы выбрали из предыдущего дз)
2. скачиваем готовый новостной датасет
!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

        ```
        # пример работы с ним 
        from corus import load_ods_gazeta
        path = 'gazeta.csv.gz'
        records = load_ods_gazeta(path)
        next(records)
        ```

    реализовать метод поиска ближайших статей
    (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру топ 5-ть или 3-ри, ваш метод должен возвращать топ-k ближайших статей к этому запросу)
    визуально оценить качество

In [1]:
import re
import io

#устанавливаем библиотеку transformers
!pip install transformers
import transformers
#для загрузки токенайзера и модели от facebook
!pip install sacremoses
import sacremoses

!pip install corus
import corus

!pip install annoy
import annoy

!pip install sentence_transformers
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 6.2 MB/s 
     |████████████████████████████████| 101 kB 12.4 MB/s 
     |████████████████████████████████| 596 kB 49.9 MB/s 
     |████████████████████████████████| 6.6 MB 51.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 7.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=b0398ac84db8ac5111349a32fc986e3007cc2cc138fcb92627c19e50401664cb
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.d

In [3]:
# Используем датасет http://www.manythings.org/anki/
!wget http://www.manythings.org/anki/rus-eng.zip

--2022-08-07 10:38:34--  http://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14819554 (14M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]  14.13M  15.7MB/s    in 0.9s    

2022-08-07 10:38:35 (15.7 MB/s) - ‘rus-eng.zip’ saved [14819554/14819554]



In [4]:
# Загружаем файл
path_to_file = "/content/rus-eng/rus.txt"

In [5]:
#Создаем папку и извлекаем туда файлы из архива
!mkdir rus-eng
!unzip rus-eng.zip -d rus-eng/

Archive:  rus-eng.zip
  inflating: rus-eng/rus.txt         
  inflating: rus-eng/_about.txt      


In [6]:
!ls /content/rus-eng/ -lah

total 71M
drwxr-xr-x 2 root root 4.0K Aug  7 10:38 .
drwxr-xr-x 1 root root 4.0K Aug  7 10:38 ..
-rw-r--r-- 1 root root 1.5K May  2 01:29 _about.txt
-rw-r--r-- 1 root root  71M May  2 01:29 rus.txt


In [7]:
#просмотр файла
f = open(path_to_file)
for line in f:
    print(line)
    break

Go.	Марш!	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1159202 (shanghainese)



In [8]:
#функция препроцессинга
def preprocess_sentence(w):
  #переводим предложение к нижнему регистру и удалем начальные и конечные пробелы
    w = w.lower().strip()

  # отделяем пробелом слово и следующую за ним пунктуацию
  # пример: "he is a boy." => "he is a boy ."
    w = re.sub(r"([?.!,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

  # все, кроме букв и знаков пунктуации, заменяем пробелом
    w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)
  
  #удаляем лишние пробелы в начале и конце
    w = w.strip()

    return w

In [9]:
def create_dataset(path, num_examples):
  #считываем строки файла
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
  #каждую строку разделяем на пробелы, берем первые 2 слова, препроцессим их и возвращаем пару
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

    return zip(*word_pairs)

In [10]:
#пример работы
en, ru = create_dataset(path_to_file, None)
print(en[1])
print(ru[1])

go .
иди .


In [11]:
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
mname = "facebook/wmt19-ru-en"
#создаем экземпляр токенайзера
tokenizer = FSMTTokenizer.from_pretrained(mname)
#создаем экземпляр модели
model = FSMTForConditionalGeneration.from_pretrained(mname)

In [12]:
#пример работы с датасетом
for i in range(10):
    input = ru[i]
  #индексы токенов
    input_ids = tokenizer.encode(input, return_tensors="pt")
  #ивыход модели
    output = model.generate(input_ids)
  #генерация текста
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f'ru: {input}, en: {decoded}, real: {en[i]}')

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


ru: марш !, en: march!, real: go .


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


ru: иди ., en: Go., real: go .
ru: идите ., en: Go., real: go .
ru: здравствуйте ., en: Hello., real: hi .
ru: привет !, en: Hello!, real: hi .
ru: хай ., en: Hai., real: hi .
ru: здрасте ., en: , real: hi .
ru: здоро во !, en: Good morning and welcome!, real: hi .
ru: приветик !, en: Hello!, real: hi .
ru: беги !, en: Run!, real: run !


In [13]:
#входные данные из домашнего задания 10
inputs = ['хорошая погода', 'я иду в школу', 'Я иду на урок в школу', 
          'Я сам решаю трудную задачу в школе', 'Я не люблю, когда идет снег.', 
          'Сегодня я пойду гулять, а потом пойду в магазин', 'Мама приготовила вкусный обед для нашей большой семьи',
          'Мне здесь нравится, но я очень скучаю по дому. Там остались мама и папа.']
for input in inputs:
  #индексы токенов
    input_ids = tokenizer.encode(input, return_tensors="pt")
  #ивыход модели
    output = model.generate(input_ids)
  #генерация текста
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f'ru: {input}, en: {decoded}')

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


ru: хорошая погода, en: good weather
ru: я иду в школу, en: I'm going to school
ru: Я иду на урок в школу, en: I go to school for a lesson
ru: Я сам решаю трудную задачу в школе, en: I solve a difficult task at school myself
ru: Я не люблю, когда идет снег., en: I don't like it when it's snowing.
ru: Сегодня я пойду гулять, а потом пойду в магазин, en: Today I'll go for a walk and then I'll go to the store
ru: Мама приготовила вкусный обед для нашей большой семьи, en: Mum cooked a delicious lunch for our big family
ru: Мне здесь нравится, но я очень скучаю по дому. Там остались мама и папа., en: I love it here, but I miss the house very much. My mum and dad stayed there.


Метод поиска ближайших соседей на примере новостного датасета 

In [14]:
#скачиваем новостной датасет 
!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

--2022-08-07 10:40:53--  https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-08-07 10:40:53 ERROR 404: Not Found.

